# Sky map demo 

This is from [Viyaleta Apgar](https://github.com/viyaleta/Python-Demos/blob/main/Sky%20Map%20Demo.ipynb) who made this sky map. From this, we can incorporate the code into a useful observing map!

In [2]:
# imports
from datetime import datetime
from geopy import Nominatim
from tzwhere import tzwhere
from pytz import timezone, utc

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib.patches import Circle

from skyfield.api import Star, load, wgs84
from skyfield.data import hipparcos
from skyfield.projections import build_stereographic_projection

In [3]:
# load celestial data

# de421 shows position of earth and sun in space
eph = load('de421.bsp') # <-- could write a function which takes into account the time and location??

# hipparcos dataset contains star location data
with load.open(hipparcos.URL) as f:
    stars = hipparcos.load_dataframe(f)

[#################################] 100% de421.bsp
[#################################] 100% hip_main.dat


In [4]:
location = 'Times Square, New York, NY' # <-- set these as an astropy or timeanddate datetime object instead
when = '2023-01-01 00:00'

In [5]:
# get latitude and longitude of our location 
locator = Nominatim(user_agent='myGeocoder') # <-- use astropy instead
location = locator.geocode(location)
lat, long = location.latitude, location.longitude

In [6]:
# convert date string into datetime object
dt = datetime.strptime(when, '%Y-%m-%d %H:%M') # <-- it literally becomes datetime anyway in a bit!!

# define datetime and convert to utc based on our timezone
timezone_str = tzwhere.tzwhere().tzNameAt(lat, long)
local = timezone(timezone_str)

# get UTC from local timezone and datetime
local_dt = local.localize(dt, is_dst=None)
utc_dt = local_dt.astimezone(utc) # <-- I have already done this in the staralt code!

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (1, 2) + inhomogeneous part.

In [7]:
# find location of earth and sun and set the observer position
sun = eph['sun'] # <-- from function instead?
earth = eph['earth']

# define observation time from our UTC datetime
ts = load.timescale() # <-- I did this manually in the staralt code
t = ts.from_datetime(utc_dt)

# define an observer using the world geodetic system data
observer = wgs84.latlon(latitude_degrees=lat, longitude_degrees=long).at(t) # <-- done with EarthLocation in astropy

# define the position in the sky where we will be looking
position = observer.from_altaz(alt_degrees=90, az_degrees=0)

NameError: name 'utc_dt' is not defined